In [ ]:
import pygame
import random
from typing import List, Tuple, Optional
import math

# =========================
# KONFIG (windowed)
# =========================
FPS = 60
TOP_GAP = 56          # tinggi top bar
BOTTOM_H = 80         # tinggi bottom panel
MARGIN = 3            # jarak antar sel
CELL_SIZE = 64        # ukuran sel
ANIM_DURATION_SEC = 0.15
AI_DELAY_MS = 400

# Level list: (name, grid_size, coins)
LEVELS = [
    ("Easy",   7, 13),
    ("Medium", 8, 13),
    ("Hard",  10, 15),
]

# =========================
# WARNA
# =========================
BG = (20, 20, 24)
CELL_BG = (40, 40, 48)

COIN_COLOR   = (240, 210, 60)
COIN_OUTLINE = (180, 150, 40)
COIN_SHINE   = (255, 255, 220)

GOLD = (255, 215, 0)   # warna emas

HUMAN_COLOR = (90, 200, 250)    # biru
AI_COLOR    = (255, 100, 120)   # merah
SHADOW_COLOR = (25, 25, 28)

TEXT    = (235, 235, 240)
SUBTEXT = (190, 195, 205)
TOPBAR_BG  = (30, 32, 38)
PANEL_BG   = (30, 30, 38)
BLACK = (0, 0, 0)

WIN_COLORS = [(255,100,120), (90,200,250), (240,210,60), (100,255,100), (255,150,255)]

# =========================
# VAR GLOBAL BERUBAH LEVEL
# =========================
WIDTH = 0
HEIGHT = 0
GRID_SIZE = 7
LEVEL_COINS = 13

def apply_level(grid_size: int, coins: int):
    """Set ukuran board & coins, hitung ulang window size."""
    global GRID_SIZE, LEVEL_COINS, WIDTH, HEIGHT
    GRID_SIZE = grid_size
    LEVEL_COINS = coins
    board_w = GRID_SIZE * (CELL_SIZE + MARGIN) + MARGIN
    WIDTH  = board_w
    HEIGHT = TOP_GAP + board_w + BOTTOM_H

# =========================
# TIPE
# =========================
Vec2  = Tuple[int, int]
Vec2f = Tuple[float, float]

# =========================
# UTIL
# =========================
def clamp(n, lo, hi): return max(lo, min(hi, n))

def manhattan(a: Vec2, b: Vec2) -> int:
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def is_adjacent(a: Vec2, b: Vec2) -> bool:
    return manhattan(a, b) == 1

def lerp(a: float, b: float, t: float) -> float:
    return a + (b - a) * t

def nearest_coin(from_pos: Vec2, coins: List[Vec2]) -> Optional[Vec2]:
    if not coins:
        return None
    return min(coins, key=lambda c: manhattan(from_pos, c))

# Ukuran board (px) dan origin agar center
def board_size_px() -> int:
    return GRID_SIZE * (CELL_SIZE + MARGIN) + MARGIN

def board_origin() -> Tuple[int, int]:
    bw = board_size_px()
    ox = (WIDTH  - bw) // 2
    oy = TOP_GAP + (HEIGHT - TOP_GAP - BOTTOM_H - bw) // 2
    return ox, oy

# Grid<->Pixel
def grid_to_px(cell: Vec2) -> Vec2f:
    x, y = cell
    ox, oy = board_origin()
    px = ox + MARGIN + x * (CELL_SIZE + MARGIN)
    py = oy + MARGIN + y * (CELL_SIZE + MARGIN)
    return (px, py)

def cell_from_mouse(mx: int, my: int) -> Optional[Vec2]:
    ox, oy = board_origin()
    bw = board_size_px()
    if not (ox <= mx < ox + bw and oy <= my < oy + bw):
        return None
    gx = (mx - ox - MARGIN) // (CELL_SIZE + MARGIN)
    gy = (my - oy - MARGIN) // (CELL_SIZE + MARGIN)
    if 0 <= gx < GRID_SIZE and 0 <= gy < GRID_SIZE:
        return (int(gx), int(gy))
    return None

def step_towards(src: Vec2, dst: Vec2) -> Vec2:
    x, y = src
    dx = dst[0] - x
    dy = dst[1] - y
    if abs(dx) >= abs(dy):
        x += 1 if dx > 0 else (-1 if dx < 0 else 0)
    else:
        y += 1 if dy > 0 else (-1 if dy < 0 else 0)
    x = clamp(x, 0, GRID_SIZE - 1)
    y = clamp(y, 0, GRID_SIZE - 1)
    return (x, y)

# =========================
# CLASS ROUND (1 level)
# =========================
class RoundGame:
    """
    State: HUMAN_TURN, HUMAN_MOVING, AI_TURN, AI_MOVING, OVER
    """
    def __init__(self):
        self.reset()

    @property
    def human_start(self) -> Vec2: return (0, GRID_SIZE - 1)
    @property
    def ai_start(self)    -> Vec2: return (GRID_SIZE - 1, 0)

    def _spawn_coins(self) -> List[Vec2]:
        taken = {self.human, self.ai}
        coins = set()
        while len(coins) < LEVEL_COINS:
            c = (random.randrange(GRID_SIZE), random.randrange(GRID_SIZE))
            if c not in taken and c not in coins:
                coins.add(c)
        return sorted(list(coins))

    def reset(self):
        # spawn awal (pastikan berbeda)
        self.human = self.human_start
        self.ai    = self.ai_start
        if self.human == self.ai:
            # seandainya grid kecil ekstrem—acak ulang AI hingga beda
            while True:
                candidate = (random.randrange(GRID_SIZE), random.randrange(GRID_SIZE))
                if candidate != self.human:
                    self.ai = candidate
                    break

        # posisi pixel (anim)
        self.human_px = grid_to_px(self.human)
        self.ai_px    = grid_to_px(self.ai)
        self.human_start_px = self.human_px
        self.ai_start_px    = self.ai_px
        self.human_target_grid = self.human
        self.ai_target_grid    = self.ai

        self.anim_timer = 0.0

        self.coins: List[Vec2] = self._spawn_coins()
        self.human_score = 0
        self.ai_score = 0

        self.state = "HUMAN_TURN"
        self.over = False
        self.result = None          # "HUMAN_WIN" / "AI_WIN" / "DRAW"

    # ---- logika skor/akhir ----
    def _collect_if_any(self, who: str):
        pos = self.human if who == "HUMAN" else self.ai
        if pos in self.coins:
            self.coins.remove(pos)
            if who == "HUMAN": self.human_score += 1
            else:              self.ai_score += 1

    def _check_over(self):
        if not self.coins:
            self.over = True
            if self.human_score > self.ai_score: self.result = "HUMAN_WIN"
            elif self.ai_score > self.human_score: self.result = "AI_WIN"
            else: self.result = "DRAW"

    # ---- Human & AI move (dengan collision handling) ----
    def human_try_move(self, target_cell: Vec2) -> bool:
        """
        Mulai animasi langkah Human ke target_cell jika:
        - state = HUMAN_TURN
        - target masih di dalam board
        - target adalah sel tetangga
        - target TIDAK ditempati AI (error handling)
        """
        if self.state != "HUMAN_TURN":
            return False
        if not (0 <= target_cell[0] < GRID_SIZE and 0 <= target_cell[1] < GRID_SIZE):
            return False
        if not is_adjacent(self.human, target_cell):
            return False
        if target_cell == self.ai:
            # tidak boleh menabrak/memasuki sel AI
            return False

        self.human_start_px = self.human_px
        self.human_target_grid = target_cell
        self.anim_timer = 0.0
        self.state = "HUMAN_MOVING"
        return True

    def ai_start_move(self):
        if self.state != "AI_TURN" or self.over:
            return
        if not self.coins:
            self._check_over(); return

        target_coin = nearest_coin(self.ai, self.coins)
        if target_coin is None:
            self._check_over(); return

        # rencana langkah 1 sel menuju koin
        candidate = step_towards(self.ai, target_coin)

        # === ERROR HANDLING POSISI ===
        # AI tidak boleh masuk sel Human
        if candidate == self.human:
            # coba alternatif: memprioritaskan sumbu lain
            ax, ay = self.ai
            tx, ty = target_coin
            options: List[Vec2] = []
            # dua kemungkinan axis
            if tx != ax:
                options.append((ax + (1 if tx > ax else -1), ay))
            if ty != ay:
                options.append((ax, ay + (1 if ty > ay else -1)))
            # filter valid dan bukan posisi human
            options = [(x, y) for (x, y) in options
                       if 0 <= x < GRID_SIZE and 0 <= y < GRID_SIZE and (x, y) != self.human]
            # jika ada opsi aman, pilih yang paling mendekatkan koin
            if options:
                candidate = min(options, key=lambda p: manhattan(p, target_coin))
            else:
                # tidak ada langkah aman—AI skip turn
                return

        self.ai_start_px = self.ai_px
        self.ai_target_grid = candidate
        self.anim_timer = 0.0
        self.state = "AI_MOVING"

    def update_animation(self, dt_sec: float):
        if self.state == "HUMAN_MOVING":
            self.anim_timer += dt_sec
            t = min(1.0, self.anim_timer / ANIM_DURATION_SEC)
            start_px = self.human_start_px
            end_px   = grid_to_px(self.human_target_grid)
            self.human_px = (lerp(start_px[0], end_px[0], t), lerp(start_px[1], end_px[1], t))
            if t >= 1.0:
                # commit grid
                self.human = self.human_target_grid
                self.human_px = end_px
                self._collect_if_any("HUMAN")
                self._check_over()
                if not self.over:
                    self.state = "AI_TURN"

        elif self.state == "AI_MOVING":
            self.anim_timer += dt_sec
            t = min(1.0, self.anim_timer / ANIM_DURATION_SEC)
            start_px = self.ai_start_px
            end_px   = grid_to_px(self.ai_target_grid)
            self.ai_px = (lerp(start_px[0], end_px[0], t), lerp(start_px[1], end_px[1], t))
            if t >= 1.0:
                self.ai = self.ai_target_grid
                self.ai_px = end_px
                self._collect_if_any("AI")
                self._check_over()
                if not self.over:
                    self.state = "HUMAN_TURN"

# =========================
# RENDER
# =========================
def draw_topbar(screen, font, font_small, level_name: str):
    pygame.draw.rect(screen, TOPBAR_BG, (0, 0, WIDTH, TOP_GAP))
    lbl = font.render(f"Round – {level_name}", True, TEXT)
    screen.blit(lbl, (16, TOP_GAP//2 - lbl.get_height()//2))

    # Legend kanan
    hx = WIDTH - 260; hy = TOP_GAP//2
    pygame.draw.circle(screen, HUMAN_COLOR, (hx, hy), 10)
    lab_h = font_small.render("Human", True, TEXT)
    screen.blit(lab_h, (hx + 14, hy - lab_h.get_height()//2))

    ax = WIDTH - 140; ay = TOP_GAP//2
    pygame.draw.rect(screen, AI_COLOR, (ax-10, ay-10, 20, 20), border_radius=4)
    lab_ai = font_small.render("AI", True, TEXT)
    screen.blit(lab_ai, (ax + 14, ay - lab_ai.get_height()//2))

def draw_grid(screen):
    screen.fill(BG)
    ox, oy = board_origin()
    for y in range(GRID_SIZE):
        for x in range(GRID_SIZE):
            px = ox + MARGIN + x * (CELL_SIZE + MARGIN)
            py = oy + MARGIN + y * (CELL_SIZE + MARGIN)
            pygame.draw.rect(screen, CELL_BG, pygame.Rect(px, py, CELL_SIZE, CELL_SIZE), border_radius=4)

def draw_coins(screen, coins: List[Vec2], bob_offset: float):
    for c in coins:
        px, py = grid_to_px(c)
        cx = px + CELL_SIZE // 2
        cy = py + CELL_SIZE // 2 + bob_offset
        r = CELL_SIZE // 4
        pygame.draw.circle(screen, COIN_OUTLINE, (cx, cy), r + 2)
        pygame.draw.circle(screen, COIN_COLOR,  (cx, cy), r)
        shine_r = r // 2
        pygame.draw.circle(screen, COIN_SHINE, (cx - shine_r//2, cy - shine_r//2), shine_r)

def draw_human(screen, px_pos: Vec2f):
    px, py = px_pos
    cx = px + CELL_SIZE // 2
    cy = py + CELL_SIZE // 2
    r = CELL_SIZE // 2 - 8
    pygame.draw.circle(screen, SHADOW_COLOR, (cx, cy + 4), r)
    pygame.draw.circle(screen, HUMAN_COLOR, (cx, cy), r)
    eye_r = CELL_SIZE // 12
    eye_y = cy - r // 3
    eye_x_off = r // 2
    pygame.draw.circle(screen, BLACK, (cx - eye_x_off, eye_y), eye_r)
    pygame.draw.circle(screen, BLACK, (cx + eye_x_off, eye_y), eye_r)
    smile_y = cy + r // 3
    smile_w = int(r / 1.5)
    pygame.draw.line(screen, BLACK, (cx - smile_w // 2, smile_y), (cx + smile_w // 2, smile_y), 4)

def draw_ai(screen, px_pos: Vec2f):
    px, py = px_pos
    pad = CELL_SIZE // 8
    shadow = pygame.Rect(px + pad, py + pad + 4, CELL_SIZE - 2*pad, CELL_SIZE - 2*pad)
    pygame.draw.rect(screen, SHADOW_COLOR, shadow, border_radius=4)
    rect = pygame.Rect(px + pad, py + pad, CELL_SIZE - 2*pad, CELL_SIZE - 2*pad)
    pygame.draw.rect(screen, AI_COLOR, rect, border_radius=4)
    eye = CELL_SIZE // 6; ep = CELL_SIZE // 6
    ey = py + pad + ep; ex1 = px + pad + ep; ex2 = px + CELL_SIZE - pad - ep - eye
    pygame.draw.rect(screen, BLACK, (ex1, ey, eye, eye))
    pygame.draw.rect(screen, BLACK, (ex2, ey, eye, eye))
    my = py + CELL_SIZE - pad - ep
    pygame.draw.line(screen, BLACK, (ex1, my), (ex2 + eye, my), 4)

def draw_panel(screen, font_big, font_small, rg: RoundGame):
    py = HEIGHT - BOTTOM_H
    pygame.draw.rect(screen, PANEL_BG, (0, py, WIDTH, BOTTOM_H))
    # skor
    t_h = font_big.render(f"Human: {rg.human_score}", True, TEXT)
    screen.blit(t_h, (16, py + 10))
    t_a = font_big.render(f"AI: {rg.ai_score}", True, TEXT)
    screen.blit(t_a, (WIDTH - t_a.get_width() - 16, py + 10))

    # turn / hasil
    if not rg.over:
        if rg.state in ("HUMAN_TURN", "HUMAN_MOVING"):
            col = HUMAN_COLOR; label = "Your Turn"
        elif rg.state in ("AI_TURN", "AI_MOVING"):
            col = AI_COLOR; label = "AI Turn"
        else:
            col = SUBTEXT; label = "—"
        big = font_big.render(label, True, col)
        screen.blit(big, (WIDTH//2 - big.get_width()//2, py + 8))
    else:
        label = "You Win!" if rg.result=="HUMAN_WIN" else ("You Lose" if rg.result=="AI_WIN" else "Draw")
        col = HUMAN_COLOR if rg.result=="HUMAN_WIN" else (AI_COLOR if rg.result=="AI_WIN" else SUBTEXT)
        big = font_big.render(label, True, col)
        screen.blit(big, (WIDTH//2 - big.get_width()//2, py + 8))

    hint = font_small.render("Use Arrow/WASD to move   •   Press R to Restart", True, SUBTEXT)
    screen.blit(hint, (WIDTH//2 - hint.get_width()//2, py + 46))

# =========================
# MENU & END SCREENS
# =========================
def draw_button(screen, rect: pygame.Rect, text: str, font, hover: bool):
    bg = (70,74,86) if hover else (50,54,62)
    pygame.draw.rect(screen, bg, rect, border_radius=10)
    lbl = font.render(text, True, TEXT)
    screen.blit(lbl, (rect.centerx - lbl.get_width()//2, rect.centery - lbl.get_height()//2))

def draw_menu(screen, font_title, font):
    screen.fill(BG)
    # GOLD = (255, 215, 0)  

    t = font_title.render("COIN HUNT", True, GOLD)
    screen.blit(t, (WIDTH//2 - t.get_width()//2, HEIGHT//3 - 40))
    s = font.render("Get Ready to Start", True, SUBTEXT)
    screen.blit(s, (WIDTH//2 - s.get_width()//2, HEIGHT//3 + 24))
    w,h = 260, 56
    rect = pygame.Rect(WIDTH//2 - w//2, HEIGHT//2, w, h)
    hover = rect.collidepoint(pygame.mouse.get_pos())
    draw_button(screen, rect, "Click to Begin", font, hover)
    return rect

def draw_next_round(screen, font_big, font):
    screen.fill(BG)
    t = font_big.render("Round Won!", True, HUMAN_COLOR)
    screen.blit(t, (WIDTH//2 - t.get_width()//2, HEIGHT//3 - 24))
    s = font.render("Continue to next round", True, TEXT)
    screen.blit(s, (WIDTH//2 - s.get_width()//2, HEIGHT//3 + 24))
    w,h = 320,56
    rect = pygame.Rect(WIDTH//2 - w//2, HEIGHT//2, w, h)
    hover = rect.collidepoint(pygame.mouse.get_pos())
    draw_button(screen, rect, "Next Round", font, hover)
    return rect

def draw_end(screen, font_big, font, win: bool):
    screen.fill(BG)
    title = "You beat all rounds!" if win else "You lost the game"
    col = HUMAN_COLOR if win else AI_COLOR
    t = font_big.render(title, True, col)
    screen.blit(t, (WIDTH//2 - t.get_width()//2, HEIGHT//3 - 24))
    s = font.render("Play Again", True, TEXT)
    screen.blit(s, (WIDTH//2 - s.get_width()//2, HEIGHT//3 + 24))
    w,h = 220,56
    rect = pygame.Rect(WIDTH//2 - w//2, HEIGHT//2, w, h)
    hover = rect.collidepoint(pygame.mouse.get_pos())
    draw_button(screen, rect, "Play Again", font, hover)
    return rect

# =========================
# MAIN
# =========================
def main():
    pygame.init()
    random.seed()

    # mulai dari Easy
    level_idx = 0
    name, size, coins = LEVELS[level_idx]
    apply_level(size, coins)

    # buat window untuk level awal
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    pygame.display.set_caption("COIN HUNT — Human vs AI")

    # fonts
    font_title = pygame.font.SysFont("bahnschrift", 48)
    font_big   = pygame.font.SysFont("bahnschrift", 32)
    font       = pygame.font.SysFont("bahnschrift", 22)
    font_small = pygame.font.SysFont("bahnschrift", 18)

    clock = pygame.time.Clock()
    running = True

    # state global: MENU, PLAYING, ROUND_CLEAR, GAME_WIN, GAME_LOSE
    state = "MENU"
    rg: Optional[RoundGame] = None
    ai_timer = AI_DELAY_MS
    bob_timer = 0.0

    while running:
        dt_ms = clock.tick(FPS)
        dt = dt_ms / 1000.0
        bob_timer = (bob_timer + dt * 5.0) % (math.pi * 2)
        bob_offset = math.sin(bob_timer) * 3.0

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            # --- Start & Flow antar state ---
            elif state == "MENU" and event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                # mulai dari Easy
                level_idx = 0
                name, size, coins = LEVELS[level_idx]
                apply_level(size, coins)
                screen = pygame.display.set_mode((WIDTH, HEIGHT))
                rg = RoundGame()
                state = "PLAYING"

            elif state == "ROUND_CLEAR" and event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                # lanjut level berikutnya
                level_idx += 1
                name, size, coins = LEVELS[level_idx]
                apply_level(size, coins)
                screen = pygame.display.set_mode((WIDTH, HEIGHT))
                rg = RoundGame()
                state = "PLAYING"

            elif state in ("GAME_WIN", "GAME_LOSE") and event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                # kembali ke MENU
                state = "MENU"

            # --- Input saat bermain ---
            elif state == "PLAYING" and event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    running = False
                elif event.key == pygame.K_r:
                    rg = RoundGame()
                elif rg and rg.state == "HUMAN_TURN":
                    cx, cy = rg.human
                    if event.key in (pygame.K_w, pygame.K_UP):
                        rg.human_try_move((cx, cy - 1))
                    elif event.key in (pygame.K_s, pygame.K_DOWN):
                        rg.human_try_move((cx, cy + 1))
                    elif event.key in (pygame.K_a, pygame.K_LEFT):
                        rg.human_try_move((cx - 1, cy))
                    elif event.key in (pygame.K_d, pygame.K_RIGHT):
                        rg.human_try_move((cx + 1, cy))

            elif state == "PLAYING" and event.type == pygame.MOUSEBUTTONDOWN and event.button == 1:
                if rg and rg.state == "HUMAN_TURN":
                    cell = cell_from_mouse(*event.pos)
                    if cell is not None:
                        rg.human_try_move(cell)

        # === UPDATE ===
        if state == "PLAYING" and rg:
            # AI timer
            if rg.state == "AI_TURN" and not rg.over:
                ai_timer -= dt_ms
                if ai_timer <= 0:
                    rg.ai_start_move()
                    ai_timer = AI_DELAY_MS
            elif rg.state != "AI_TURN":
                ai_timer = AI_DELAY_MS

            rg.update_animation(dt)

            # round selesai?
            if rg.over:
                if rg.result == "HUMAN_WIN":
                    if level_idx == len(LEVELS) - 1:
                        state = "GAME_WIN"
                    else:
                        state = "ROUND_CLEAR"
                else:
                    state = "GAME_LOSE"

        # === DRAW ===
        if state == "MENU":
            draw_menu(screen, font_title, font)

        elif state == "PLAYING" and rg:
            draw_grid(screen)
            draw_topbar(screen, font, font_small, LEVELS[level_idx][0])
            draw_coins(screen, rg.coins, bob_offset)
            draw_human(screen, rg.human_px)
            draw_ai(screen, rg.ai_px)
            draw_panel(screen, font_big, font_small, rg)

        elif state == "ROUND_CLEAR":
            draw_next_round(screen, font_big, font)

        elif state == "GAME_WIN":
            draw_end(screen, font_big, font, True)

        elif state == "GAME_LOSE":
            draw_end(screen, font_big, font, False)

        pygame.display.flip()

    pygame.quit()

if __name__ == "__main__":
    # hitung ukuran awal window sesuai Easy
    apply_level(LEVELS[0][1], LEVELS[0][2])
    main()
